In [1]:
!pip install --quiet img2vec_pytorch
print('pip installed img2vec')

from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning) # quiet a plotly issue
filterwarnings(action='ignore', category=UserWarning) # quiet an img2vec issue

pip installed img2vec


In [2]:
from img2vec_pytorch import Img2Vec
from PIL import Image
from arrow import now
from glob import glob
import pandas as pd
from os.path import basename

SIZE = 512
STOP = 10000 
DATA_GLOB = '/kaggle/input/rock-paper-scissors-dataset'

# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

def get_from_glob(arg: str, tag: str, stop: int) -> list:
    time_get = now()
    result = []
    for index, input_file in enumerate(glob(pathname=arg)):
        if index < stop:
            name = basename(input_file)
            with Image.open(fp=input_file, mode='r') as image:
                vector = img2vec.get_vec(image, tensor=True).numpy().reshape(SIZE,)
                result.append(pd.Series(data=[tag, name, vector], index=['tag', 'name', 'value']))
    print('encoded {} data {} rows in {}'.format(tag, len(result), now() - time_get))
    return result


img2vec = Img2Vec(cuda=False, model='resnet-18', layer='default', layer_output_size=SIZE)

time_start = now()

files = {basename(folder) : folder + '/*.png' for folder in glob(DATA_GLOB + '/*')}
data = [get_from_glob(arg=value, tag=key, stop=STOP) for key, value in files.items()]
df = pd.DataFrame(data=flatten(arg=data))
    
print('done in {}'.format(now() - time_start))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 57.1MB/s]


encoded paper data 907 rows in 0:01:01.833207
encoded rock data 907 rows in 0:01:01.986712
encoded scissors data 903 rows in 0:01:02.135709
done in 0:03:06.232255


Our classes are balanced and we have a small number of classes. Let's visualize using UMAP.

In [3]:
from arrow import now
from umap import UMAP

time_start = now()
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=500)
plot_df = pd.concat(objs=[df, pd.DataFrame(data=umap.fit_transform(X=df['value'].apply(pd.Series)), columns=['x', 'y'])], axis=1)
print('done with UMAP in {}'.format(now() - time_start))

2024-02-24 15:00:48.647543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 15:00:48.647800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 15:00:48.850232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=500, n_jobs=1, random_state=2024, verbose=True)
Sat Feb 24 15:01:03 2024 Construct fuzzy simplicial set
Sat Feb 24 15:01:12 2024 Finding Nearest Neighbors
Sat Feb 24 15:01:17 2024 Finished Nearest Neighbor Search
Sat Feb 24 15:01:21 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Sat Feb 24 15:01:27 2024 Finished embedding
done with UMAP in 0:00:25.011223


In [4]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='x', y='y', color='tag', hover_name='name', height=900).show()

This is an odd looking clustering, in that we appear to have four clusters that have smaller subclusters within them, but the different classes seem to be tightly clustered inside them; we don't have regions in X-Y space that have a mix of all three classes. So on this basis we expect even a simple model will do a good job classifying these.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(df['value'].apply(pd.Series), df['tag'], test_size=0.25, random_state=2024)
model = LogisticRegression(max_iter=100000)
model.fit(X_train, y_train)

print('accuracy: {:5.2f} pct'.format(100 * accuracy_score(y_test, model.predict(X_test))))

accuracy: 99.26 pct


In [6]:
from sklearn.metrics import classification_report

print(classification_report(labels=df['tag'].unique().tolist(), y_true=y_test, y_pred=model.predict(X_test)))


              precision    recall  f1-score   support

       paper       1.00      0.99      0.99       223
        rock       0.99      1.00      1.00       226
    scissors       0.99      0.99      0.99       231

    accuracy                           0.99       680
   macro avg       0.99      0.99      0.99       680
weighted avg       0.99      0.99      0.99       680

